# 使用真实数据测试sgd

In [14]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [15]:
from myscript.train_test_split import train_test_split

In [16]:
X_train, y_train, X_test, y_test = train_test_split(X, y, seed=666)

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [19]:
X_train_standard = standardScaler.transform(X_train)

In [20]:
X_test_standard = standardScaler.transform(X_test)

In [21]:
from myscript.LinearRegression import LinearRegression

In [22]:
lin_reg = LinearRegression()

In [23]:
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=2)

Wall time: 6.98 ms


LinearRegression()

In [24]:
lin_reg.score(X_test_standard, y_test)

0.7857275413602651

In [25]:
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=50)
lin_reg.score(X_test_standard, y_test)

Wall time: 124 ms


0.808560757055621

# sklearn中sgd

In [29]:
from sklearn.linear_model import SGDRegressor

In [30]:
sgd_reg = SGDRegressor()

In [33]:
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

Wall time: 7.98 ms


0.8123287969388414

In [34]:
np.random.seed(666)

In [36]:
X = np.random.random(size= (1000, 10))

In [37]:
true_theta = np.arange(1, 12,  dtype=float)

In [39]:
X_b = np.hstack([np.ones((len(X), 1)), X])

In [40]:
y = X_b.dot(true_theta)+ np.random.normal(size=1000)

In [41]:
def J(theta, X_b, y):
    try:
        return np.sum((y-X_b.dot(theta))**2)/ len(X_b)
    except:
        return float('inf') 

In [43]:
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta)-y)*2. / len(X_b)

In [51]:
def dJ_debug(theta, X_b, y, epsilon = 0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta1 = theta.copy()
        theta1[i] += epsilon
        theta2 = theta.copy()
        theta2[i] -= epsilon
        res[i] = (J(theta1, X_b, y)-J(theta2, X_b, y))/(2*epsilon)
    return res

In [52]:
def gradient_descent(dJ,  X_b, y,initial_theta, eta,n_iters=1e4, epsilon = 1e-8):
    theta = initial_theta
    i_iter = 0
    while i_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta*gradient
        if (abs(J(theta, X_b, y)-J(last_theta, X_b, y)) < epsilon):
            break
        i_iter +=1
    return theta

In [53]:
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

In [60]:
%time theta = gradient_descent(dJ_debug, X_b, y,initial_theta, eta)
theta

Wall time: 4.23 s


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

In [61]:
%time theta = gradient_descent(dJ_math, X_b, y,initial_theta, eta)
theta

Wall time: 755 ms


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

dJ_debug用来检验dJ_math，作为工具箱使用